---
title: "Annexe D : Validation du modèle"
execute:
  cache: false
---

In [ ]:
#| label: setup
#| output: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_ljungbox
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy import stats
from IPython.display import Markdown, display

# Configuration graphique
sns.set_theme(style="whitegrid", context="paper", palette="colorblind")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# ═══════════════════════════════════════════════════════════════════
# CHARGEMENT ET PRÉPARATION DES DONNÉES
# ═══════════════════════════════════════════════════════════════════

DATA_LOADED = False

try:
    df = pd.read_csv("../../data/01_raw/kuznets_data_final.csv")
    DATA_LOADED = True
    
    # ─────────────────────────────────────────────────────────────────
    # DONNÉES TRANSVERSALES
    # ─────────────────────────────────────────────────────────────────
    
    ANNEE_CROSS = int(df['year'].max())
    df_cross = df[df['year'] == ANNEE_CROSS].copy()
    
    # Exclusions (micro-États et paradis fiscaux)
    # Justification : Micro-États (<500k hab) et paradis fiscaux 
    # ont une structure économique atypique (PIB/hab artificiellement gonflé)
    EXCLUSIONS = [
        'Liechtenstein', 'Monaco', 'San Marino', 'Andorra',  # Micro-États
        'Luxembourg', 'Ireland', 'Singapore', 'Malta', 'Cyprus',  # Hubs financiers
        'Palau', 'Nauru', 'Tuvalu', 'Vatican'  # Micro-États insulaires
    ]
    
    # Nettoyage    
    df_cross = df_cross[~df_cross['country'].isin(EXCLUSIONS)]
    df_cross = df_cross.dropna(subset=['gdp_per_capita', 'co2_per_capita'])
    
    # Transformations log
    df_cross['log_gdp'] = np.log(df_cross['gdp_per_capita'])
    df_cross['log_co2'] = np.log(df_cross['co2_per_capita'])

    # Création d'un alias pratique pour l'affichage plus tard
    # (S'assure qu'on utilise bien la colonne divisée par la population)    
    df_cross['gdp_pc'] = df_cross['gdp_per_capita']
    
    # Centrage pour réduire multicolinéarité polynomiale
    GDP_MEAN = df_cross['log_gdp'].mean()
    df_cross['log_gdp_c'] = df_cross['log_gdp'] - GDP_MEAN
    
    N_PAYS_EXCLUS = len(EXCLUSIONS)
    N_PAYS = len(df_cross)
    
    # ─────────────────────────────────────────────────────────────────
    # DONNÉES TEMPORELLES (USA)
    # ─────────────────────────────────────────────────────────────────
    
    df_usa = df[df['country'] == 'United States'].sort_values('year').copy()
    
    if not df_usa.empty:
        # Donnée brute disponible dès 1990
        ANNEE_DEBUT_DATA = int(df_usa['year'].min())  # 1990

        # Transformations log      
        df_usa['log_co2'] = np.log(df_usa['co2_per_capita'])
        df_usa['log_gdp'] = np.log(df_usa['gdp_per_capita'])
        
        # Création d'un alias pratique pour l'affichage plus tard
        df_usa['gdp_pc'] = df_usa['gdp_per_capita']

        # Centrage USA, Moyenne calculée AVANT dropna pour cohérence
        GDP_MEAN_USA = df_usa['log_gdp'].mean()
        df_usa['log_gdp_c'] = df_usa['log_gdp'] - GDP_MEAN_USA

        # Création des lags (APRÈS centrage)
        df_usa['lag_log_co2'] = df_usa['log_co2'].shift(1)
        df_usa['lag_log_gdp_c'] = df_usa['log_gdp_c'].shift(1) 
        
        # Nettoyage : enlève la première observation (1990) car lag = NaN
        df_usa = df_usa.dropna()

        # Estimation démarre en 1991 (après création du lag)
        ANNEE_DEBUT_ESTIM = int(df_usa['year'].min())  # 1991
        ANNEE_FIN = int(df_usa['year'].max())          # 2020
        N_OBS_USA = len(df_usa)                        # 30 observations
    else:
        N_OBS_USA = ANNEE_DEBUT = ANNEE_FIN = 0

except FileNotFoundError:
    print("⚠️ ERREUR : Fichier data/01_raw/kuznets_data_final.csv introuvable.")
    print("   Vérifiez le chemin relatif depuis le répertoire du notebook.")
    df_cross = pd.DataFrame()
    df_usa = pd.DataFrame()
    N_PAYS = N_PAYS_EXCLUS = N_OBS_USA = 0
    ANNEE_DEBUT_DATA = ANNEE_DEBUT_ESTIM = ANNEE_FIN = ANNEE_CROSS = 0
    GDP_MEAN = GDP_MEAN_USA = 0

except ValueError as e:
    print(f"⚠️ ERREUR DE DONNÉES : {e}")
    print("   Causes possibles : valeurs négatives dans PIB ou CO₂, types incorrects.")
    df_cross = pd.DataFrame()
    df_usa = pd.DataFrame()
    N_PAYS = N_PAYS_EXCLUS = N_OBS_USA = 0
    ANNEE_DEBUT_DATA = ANNEE_DEBUT_ESTIM = ANNEE_FIN = ANNEE_CROSS = 0
    GDP_MEAN = GDP_MEAN_USA = 0

except Exception as e:
    print(f"⚠️ ERREUR INATTENDUE : {e}")
    import traceback
    traceback.print_exc()
    df_cross = pd.DataFrame()
    df_usa = pd.DataFrame()
    N_PAYS = N_PAYS_EXCLUS = N_OBS_USA = 0
    ANNEE_DEBUT_DATA = ANNEE_DEBUT_ESTIM = ANNEE_FIN = ANNEE_CROSS = 0
    GDP_MEAN = GDP_MEAN_USA = 0

## Validation de la spécification log-log {#sec-box-cox}

Le choix d'une transformation logarithmique pour la variable dépendante ($\text{CO}_2/\text{hab}$)
n'est pas arbitraire. Le **test de Box-Cox** détermine empiriquement la 
transformation optimale en estimant le paramètre $\lambda$ :

$$Y^{(\lambda)} = \begin{cases} \frac{Y^\lambda - 1}{\lambda} & \text{si } \lambda \neq 0 \\ \ln(Y) & \text{si } \lambda = 0 \end{cases}$$

In [ ]:
#| label: test-box-cox
#| echo: false

if DATA_LOADED and not df_cross.empty:
    from scipy import stats as sp_stats
    
    # Box-Cox sur CO2 per capita
    _, lambda_optimal = sp_stats.boxcox(df_cross['co2_per_capita'])
    
    distance_log = abs(lambda_optimal - 0)
    distance_lin = abs(lambda_optimal - 1)
    
    if distance_log < 0.3:
        verdict = "Fortement justifiée"
    elif distance_log < distance_lin:
        verdict = "Justifiée"
    else:
        verdict = "Alternative à considérer"
    
    display(Markdown(f"""
**Résultats du test de Box-Cox**

| Paramètre | Valeur |
|-----------|--------|
| $\lambda$ optimal estimé | {lambda_optimal:.2f} |
| Distance à log ($\lambda=0$) | {distance_log:.2f} |
| Distance à linéaire ($\lambda=1$) | {distance_lin:.2f} |
| **Verdict** | {verdict} |

**Interprétation** : 

- $\lambda = 0$ → transformation logarithmique optimale
- $\lambda = 1$ → aucune transformation (modèle linéaire)
- $\lambda = 0.5$ → transformation racine carrée

Avec $\lambda = {lambda_optimal:.3f}$, la transformation logarithmique est 
{'parfaitement' if distance_log < 0.1 else 'raisonnablement'} adaptée 
à nos données. Ce résultat valide le choix de spécification log-log 
standard dans la littérature EKC.
"""))

::: {.callout-note}
### Note technique
Le test de Box-Cox maximise la vraisemblance du modèle pour différentes 
valeurs de $\lambda$. Un $\lambda$ proche de 0 indique que la log-transformation 
maximise la normalité des résidus et stabilise la variance.
:::

## Validation non-paramétrique de la forme fonctionnelle {#sec-lowess}

Les modèles polynomiaux (quadratique, cubique) imposent une forme 
fonctionnelle *a priori*. Pour vérifier que cette hypothèse n'est 
pas trop restrictive, nous comparons les prédictions paramétriques 
à une régression **LOWESS** (*Locally Weighted Scatterplot Smoothing*), 
qui laisse les données "parler d'elles-mêmes".

In [ ]:
#| label: fig-check-models-comparisonv
#| fig-cap: Modèles paramétriques (Quadratique et Cubique) vs Modèle non-paramétrique (LOWESS)
#| warning: false

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

if 'df_cross' in locals() and not df_cross.empty:
    
    # 1. PRÉPARATION
    df_sorted = df_cross.sort_values('gdp_pc')
    
    # Modèles paramétriques
    m_quad = smf.ols('log_co2 ~ log_gdp_c + I(log_gdp_c**2)', data=df_sorted).fit()
    m_cub = smf.ols('log_co2 ~ log_gdp_c + I(log_gdp_c**2) + I(log_gdp_c**3)', data=df_sorted).fit()
    
    # Prédictions (Log)
    pred_quad_log = m_quad.predict(df_sorted)
    pred_cub_log = m_cub.predict(df_sorted)
    
    # Prédictions (Linéaire)
    pred_quad_lin = np.exp(pred_quad_log)
    pred_cub_lin = np.exp(pred_cub_log)

    # CALCUL EXPLICTE DU LOWESS (Non-Paramétrique)
    # frac=0.3 signifie qu'on lisse sur 30% des points locaux
    lowess_log = sm.nonparametric.lowess(df_sorted['log_co2'], df_sorted['log_gdp'], frac=0.3)
    lowess_lin = sm.nonparametric.lowess(df_sorted['co2_per_capita'], df_sorted['log_gdp'], frac=0.3)

    # 2. GRAPHIQUE
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # --- GAUCHE : LOG-LOG ---
    axes[0].scatter(df_sorted['log_gdp'], df_sorted['log_co2'], alpha=0.3, color='gray', label='Données')
    
    axes[0].plot(df_sorted['log_gdp'], pred_quad_log, 'g--', linewidth=2, label='Quadratique')
    axes[0].plot(df_sorted['log_gdp'], pred_cub_log, 'r-', linewidth=2.5, label='Cubique')
    
    # Tracé manuel du LOWESS (Colonne 0 = X, Colonne 1 = Y)
    axes[0].plot(lowess_log[:, 0], lowess_log[:, 1], 'b:', linewidth=3, label='Non-Paramétrique (LOWESS)')

    axes[0].set_title("A. Modèles en Log-Log (Théorie)", fontweight='bold')
    axes[0].set_xlabel("Log PIB")
    axes[0].set_ylabel("Log CO2")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # --- DROITE : LOG-LIN ---
    axes[1].scatter(df_sorted['log_gdp'], df_sorted['co2_per_capita'], alpha=0.3, color='gray', label='Données')
    
    axes[1].plot(df_sorted['log_gdp'], pred_quad_lin, 'g--', linewidth=2, label='Quadratique')
    axes[1].plot(df_sorted['log_gdp'], pred_cub_lin, 'r-', linewidth=2.5, label='Cubique')
    
    # Tracé manuel du LOWESS Linéaire
    axes[1].plot(lowess_lin[:, 0], lowess_lin[:, 1], 'b:', linewidth=3, label='Non-Paramétrique (LOWESS)')

    axes[1].set_title("B. Modèles en Tonnes (Réalité)", fontweight='bold')
    axes[1].set_xlabel("Log PIB")
    axes[1].set_ylabel("CO2 (Tonnes)")
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

else:
    print("Erreur : df_cross n'est pas chargé.")

In [ ]:
#| label: fig-quad-vs-cubic-diff
#| fig-cap: 'Test visuel du rebond : différence Quadratique vs Cubique'
#| warning: false

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

if 'df_cross' in locals() and not df_cross.empty:
    
    df_sorted = df_cross.sort_values('gdp_pc')
    
    # Modèles
    m_quad = smf.ols('log_co2 ~ log_gdp_c + I(log_gdp_c**2)', data=df_sorted).fit()
    m_cub = smf.ols('log_co2 ~ log_gdp_c + I(log_gdp_c**2) + I(log_gdp_c**3)', data=df_sorted).fit()
    
    # Prédictions
    pred_quad_log = m_quad.predict(df_sorted)
    pred_cub_log = m_cub.predict(df_sorted)
    
    # Différence pour le zoom
    diff = pred_cub_log - pred_quad_log
    
    # LOWESS
    lowess_log = sm.nonparametric.lowess(df_sorted['log_co2'], df_sorted['log_gdp'], frac=0.3)
    
    # IC 95% Quadratique
    predictions = m_quad.get_prediction(df_sorted)
    pred_summary = predictions.summary_frame(alpha=0.05)

    # GRAPHIQUE 3 PANELS
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # ======================
    # PANEL A : Superposition
    # ======================
    axes[0].scatter(df_sorted['log_gdp'], df_sorted['log_co2'], 
                   alpha=0.25, color='lightgray', s=30, label='Données (n=159)')
    
    axes[0].fill_between(df_sorted['log_gdp'], 
                        pred_summary['mean_ci_lower'], 
                        pred_summary['mean_ci_upper'],
                        alpha=0.15, color='blue', label='IC 95%')
    
    # ORDRE INVERSÉ pour voir la cubique en dessous
    axes[0].plot(df_sorted['log_gdp'], pred_cub_log, 
                'r-', linewidth=4, label='Cubique (dessous)', alpha=0.6, zorder=2)
    axes[0].plot(df_sorted['log_gdp'], pred_quad_log, 
                'b--', linewidth=2.5, label='Quadratique (dessus)', alpha=0.9, zorder=3)
    
    axes[0].set_title("A. Superposition (quasi-parfaite)", fontweight='bold', fontsize=12)
    axes[0].set_xlabel("Log PIB (centré)", fontsize=10)
    axes[0].set_ylabel("Log CO₂ (t/hab)", fontsize=10)
    axes[0].legend(loc='upper left', fontsize=9, framealpha=0.95)
    axes[0].grid(True, alpha=0.3, linestyle='--')
    
    # ======================
    # PANEL B : DIFFÉRENCE
    # ======================
    axes[1].axhline(0, color='black', linewidth=1, linestyle='--', alpha=0.5, label='Différence nulle')
    axes[1].plot(df_sorted['log_gdp'], diff, 
                'purple', linewidth=3, label='Cubique - Quadratique', alpha=0.8)
    axes[1].fill_between(df_sorted['log_gdp'], diff, 0, 
                        alpha=0.2, color='purple')
    
    # Zone hauts revenus
    axes[1].axvspan(10.8, 11.2, alpha=0.1, color='red', zorder=0)
    axes[1].text(11.0, 0.98, 'Zone critique\n(PIB > $50k)', 
                ha='center', fontsize=9, color='darkred',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.3))
    
    axes[1].set_title("B. Différence Cubique - Quadratique", fontweight='bold', fontsize=12)
    axes[1].set_xlabel("Log PIB (centré)", fontsize=10)
    axes[1].set_ylabel("Écart (log CO₂)", fontsize=10)
    axes[1].legend(loc='best', fontsize=9, framealpha=0.95)
    axes[1].grid(True, alpha=0.3, linestyle='--')
    
    # Annotation résultat
    max_diff = np.abs(diff).max()
    axes[1].text(0.02, 0.98, 
                f'Écart maximal : {max_diff:.4f}\n(≈ 0% en pratique)',
                transform=axes[1].transAxes, 
                fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgreen', alpha=0.7))
    
    # ======================
    # PANEL C : Validation LOWESS
    # ======================
    axes[2].scatter(df_sorted['log_gdp'], df_sorted['log_co2'], 
                   alpha=0.25, color='lightgray', s=30, label='Données')
    
    axes[2].plot(df_sorted['log_gdp'], pred_quad_log, 
                'b-', linewidth=3, label='Quadratique', alpha=0.8)
    axes[2].plot(lowess_log[:, 0], lowess_log[:, 1], 
                'orange', linewidth=2.5, linestyle=':', label='LOWESS', alpha=0.9)
    
    axes[2].set_title("C. Validation non-paramétrique", fontweight='bold', fontsize=12)
    axes[2].set_xlabel("Log PIB", fontsize=10)
    axes[2].set_ylabel("Log CO₂", fontsize=10)
    axes[2].legend(loc='upper left', fontsize=9, framealpha=0.95)
    axes[2].grid(True, alpha=0.3, linestyle='--')

    plt.tight_layout()
    
    plt.subplots_adjust(bottom=0.12)
    plt.show()